In [1]:
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel
from transformers import Trainer, TrainingArguments
from transformers import AdamW
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_metric

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"
print(tokenizer.tokenize(sentence1), tokenizer.tokenize(sentence2))

['what', 'a', 'beautiful', 'day', '!'] ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [3]:
inputs = tokenizer([sentence1, sentence2], padding=True)
inputs

{'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [4]:
inputs = tokenizer(sentence1, sentence2, padding=True)
inputs

{'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train_val, X_test, y_train_val, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [9]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        super().__init__()
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

In [16]:
train_dataset

In [17]:
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\admin\AppData\Local\Temp\ipykernel_7748\1064002550.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [18]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=2,
                                  per_device_train_batch_size=16, per_device_eval_batch_size=16)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, compute_metrics=compute_metrics)
trainer.train()

c:\Users\admin\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1120
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 140
  Number of trainable parameters = 109483778


  0%|          | 0/140 [00:00<?, ?it/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_7748\2252392273.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 6281.4994, 'train_samples_per_second': 0.357, 'train_steps_per_second': 0.022, 'train_loss': 0.5150371551513672, 'epoch': 2.0}


TrainOutput(global_step=140, training_loss=0.5150371551513672, metrics={'train_runtime': 6281.4994, 'train_samples_per_second': 0.357, 'train_steps_per_second': 0.022, 'train_loss': 0.5150371551513672, 'epoch': 2.0})

In [19]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 400
  Batch size = 16
C:\Users\admin\AppData\Local\Temp\ipykernel_7748\2252392273.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.3671139180660248,
 'eval_accuracy': 0.8525,
 'eval_runtime': 378.2219,
 'eval_samples_per_second': 1.058,
 'eval_steps_per_second': 0.066,
 'epoch': 2.0}

In [20]:
del model
del trainer
torch.cuda.empty_cache()

In [21]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

loading file vocab.txt from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

In [29]:
class MyModel(nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model
        
        self.classifier = nn.Linear(self.token_size, self.num_labels)
    
    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs)
        bert_clf_token = outputs.last_hidden_state[:, 0, :]
        return self.classifier(bert_clf_token)

In [30]:
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size)

In [31]:
optim = AdamW(model.parameters(), lr=1e-3)
loss_function = nn.CrossEntropyLoss()
num_epochs = 1

c:\Users\admin\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
for epoch in range(num_epochs):
    total_epoch_loss = 0

    for step, batch in enumerate(train_loader):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(inputs)

        loss = loss_function(outputs, F.one_hot(labels, num_classes=2).float())

        if (step+1) % 100 == 0:
            print(f"Epoch {epoch+1}, batch {step+1}, loss {loss:.4f}")
        
        optim.zero_grad()
        loss.backward()
        optim.step()

        total_epoch_loss += float(loss)

    avg_epoch_loss = total_epoch_loss / len(train_loader)
    print(f"Avg loss for epoch {epoch+1}: {avg_epoch_loss:.4f}")

C:\Users\admin\AppData\Local\Temp\ipykernel_7748\2252392273.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Avg loss for epoch 1: 0.8024


In [33]:
test_loader = DataLoader(test_dataset, batch_size=16)

In [34]:
metric = load_metric('accuracy')

In [35]:
model.eval()
for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(inputs)
    
    predictions = torch.argmax(outputs, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

C:\Users\admin\AppData\Local\Temp\ipykernel_7748\2252392273.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'accuracy': 0.505}